# Descripción del sistema
El sistema consiste en el desarrollo de 5 gráficos diferentes, estos gráficos son realizados con dos conjuntos de datos de dos entidades costarricenses diferentes que son el INEC y el OIJ, ambos conjuntos de datos fueron descargados en formato csv y son almacenados como dataframes utilizando la función de spark en python, además de estos los datos son limpiados ya que tienen incongruencias en su formato, una vez con los datos limpios se envían los mismo a postgres y se grafican

In [6]:
#Bibliotecas necesarias para el uso de spark
from pyspark.sql import SparkSession
from pyspark.files import SparkFiles
from pyspark.sql import functions as F
from pyspark.sql.functions import col, unix_timestamp, to_date

In [7]:
#Se guardan ambos archivos como dataframes
oij_csv=spark.read.format("csv").option("header","true").option("inferSchema","true").load("Estadisticas.csv")
inec_csv=spark.read.format("csv").option("header","true").option("inferSchema","true").load("reempleocenso2011-22.csv")

In [8]:
#Se imprimen ambos esquemas para observar el formato en el que vienen los datos, esto servirá para ajustar
#el dataframe a lo que necesitemos
#este de abajo será el formato del OIJ
oij_csv.printSchema()

root
 |-- Delito: string (nullable = true)
 |-- SubDelito: string (nullable = true)
 |-- Fecha: string (nullable = true)
 |-- Hora: string (nullable = true)
 |-- Victima: string (nullable = true)
 |-- SubVictima: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Nacionalidad: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Canton: string (nullable = true)
 |-- Distrito: string (nullable = true)



In [9]:
#Este otro formato correspone al INEC
inec_csv.printSchema()

root
 |-- ProvinciaCantonDistrito: string (nullable = true)
 |-- PoblacionMayor15: integer (nullable = true)
 |-- TasaNetaParticipacion: double (nullable = true)
 |-- TasaOcupacion: double (nullable = true)
 |-- TasaDesempleoAbierto: string (nullable = true)
 |-- PorcentajeEconomicamenteInactivo: double (nullable = true)
 |-- RelacionDependenciaEconomica: double (nullable = true)



In [10]:
#Se registran ambos dataframe como tablas de SQL, esto nos permite realizar consultas sql por medio de sqlContext
sqlContext.registerDataFrameAsTable(oij_csv, "oij")
sqlContext.registerDataFrameAsTable(inec_csv, "inec")

In [11]:
'''
Entradas: Un entero
Salidas: Un dataframe
Descripción general: Se encarga de validar cual conjunto de datos se debe enviar 
y realiza la consulta SQL correspondiente la cual removerá espacios en blanco
'''
def quitaEspacios(ind):
    #Este if es de validación para enviar datos de INEC o del OIJ
    #por medio de sqlContext se utiliza la función TRIM que acorta todos los espacios en blanco
    #almacenados antes o después de nuestros datos
    if ind == 1:
        return sqlContext.sql("SELECT Delito, SubDelito,Hora, Fecha, Victima, SubVictima, Edad, Genero, Nacionalidad, Provincia,Canton, TRIM(Distrito) as Distrito from oij")
    else:
        return sqlContext.sql("SELECT TRIM(ProvinciaCantonDistrito) AS ProvinciaCantonDistrito, PoblacionMayor15, TasaNetaParticipacion, TasaOcupacion, TasaDesempleoAbierto, PorcentajeEconomicamenteInactivo, RelacionDependenciaEconomica from inec")

In [12]:
#Simplemente se llama a la función con el número entero correspondiente y se registra el nuevo dataframe
#como la tabla almacenada anteriormente en el sistema, esto se aplica para ambos conjuntos de datos
oij = quitaEspacios(1)
sqlContext.registerDataFrameAsTable(oij, "oij")
inec = quitaEspacios(2)
sqlContext.registerDataFrameAsTable(inec, "inec")

In [13]:
'''
Entradas: Un entero
Salidas: Un dataframe
Descripción general: Se encarga de validar cual conjunto de datos se debe enviar 
y realiza la consulta SQL correspondiente la cual pasará los datos a minúscula
'''
def minusculas(ind):
    #Este if es de validación para enviar datos de INEC o del OIJ
    #por medio de sqlContext se utiliza la función LOWER que se encarga de pasar los datos a minúscula
    if ind == 1:
        return sqlContext.sql("SELECT Delito, SubDelito,Hora, Fecha, Victima, SubVictima, Edad, Genero, Nacionalidad, Provincia,Canton, LOWER(Distrito) as Distrito from oij")
    else:
        return sqlContext.sql("SELECT LOWER(ProvinciaCantonDistrito) AS ProvinciaCantonDistrito, PoblacionMayor15, TasaNetaParticipacion, TasaOcupacion, TasaDesempleoAbierto, PorcentajeEconomicamenteInactivo, RelacionDependenciaEconomica from inec")

In [14]:
#Se llama a la función con el número entero correspondiente y se registra el nuevo dataframe
#como la tabla almacenada anteriormente en el sistema, se aplica a ambos conjuntos de datos
oij = minusculas(1)
sqlContext.registerDataFrameAsTable(oij, "oij")
inec = minusculas(2)
sqlContext.registerDataFrameAsTable(inec, "inec")

In [15]:
'''
Entradas: No posee
Salidas: Un dataframe
Descripción general: Se encarga de hacer la consulta sql para mostrar los datos del OIJ
que no coinciden con ningún dato del INEC
'''
def sacaNoExistentes():
    #Esta consulta se encarga de sacar los distintos por medio del uso de DISTINCT que no se encuentren
    #en la consulta realizada a los datos del INEC
    return sqlContext.sql("SELECT DISTINCT(Distrito) FROM oij WHERE NOT EXISTS(SELECT 1 FROM inec WHERE inec.ProvinciaCantonDistrito = oij.Distrito)") 

In [16]:
#Llama a la función y lo almacena como un dataframe nuevo y una tabla nueva
noCoinciden = sacaNoExistentes()
sqlContext.registerDataFrameAsTable(noCoinciden, "noCoincidencias")

In [17]:
#Se muestran los datos que no coincidieron, principalmente fue porque el OIJ utiliza abreviaciones y no usa tildes,
#el INEC no
noCoinciden.show()

+-------------+
|     Distrito|
+-------------+
|       pococi|
|      guacimo|
|     la union|
|         null|
|  leon cortes|
|     san jose|
|    san ramon|
|        canas|
|        limon|
|perez zeledon|
|        tibas|
|    sarapiqui|
+-------------+



In [18]:
'''
Entradas: No posee
Salidas: Un dataframe
Descripción general: Se encarga de contar los elementos de la tabla de noCoincidencias
estos elementos corresponen a todos aquellos distritos del OIJ que no corresponden a distritos del INEC
'''
def cuentaNoExistentes():
    return sqlContext.sql("SELECT COUNT(*) as num FROM noCoincidencias") 

In [19]:
#Almacena el dataframe con el dato
numNoCoinciden = cuentaNoExistentes()

In [20]:
#Se despliega el número correspondiente
numNoCoinciden.show()
print("Cantidad de registros que no coincidieron: ", noCoinciden.count())

+---+
|num|
+---+
| 12|
+---+

Cantidad de registros que no coincidieron:  12


In [21]:
#Se modifican datos del dataframe del INEC para que correspondan a algunos datos del OIJ
#esto se encarga de buscar por columna los datos que tengan almaceanado los nombres de la manera indicada
#luego se modifican por sus correpondientes en el dataframe del OIJ
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='pococí','pococi').otherwise(F.col("ProvinciaCantonDistrito")))
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='la unión','la union').otherwise(F.col("ProvinciaCantonDistrito")))
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='belén','belen').otherwise(F.col("ProvinciaCantonDistrito")))
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='león cortés castro','leon cortes').otherwise(F.col("ProvinciaCantonDistrito")))
inec = inec.withColumn("ProvinciaCantonDistrito", F.when(F.col("ProvinciaCantonDistrito")=='san josé','san jose').otherwise(F.col("ProvinciaCantonDistrito")))
#se guarda el dataframe modificado como una tabla
sqlContext.registerDataFrameAsTable(inec, "inec")

In [22]:
#Se integran por medio de un inner join los conjuntos de datos,y se almacenan como un nuevo dataframe
#este dataframe será enviado a postgres
datosIntegrados = sqlContext.sql("SELECT a.Delito, a.SubDelito, a.Hora, a.Fecha, a.Victima, a.SubVictima, a.Edad, a.Genero, a.Nacionalidad, a.Provincia, a.Canton, a.Distrito, b.PoblacionMayor15, b.TasaNetaParticipacion, b.TasaOcupacion, b.TasaDesempleoAbierto, b.PorcentajeEconomicamenteInactivo, b.RelacionDependenciaEconomica FROM oij a INNER JOIN inec b ON a.Distrito = b.ProvinciaCantonDistrito")

In [23]:
datosIntegrados.printSchema()

root
 |-- Delito: string (nullable = true)
 |-- SubDelito: string (nullable = true)
 |-- Hora: string (nullable = true)
 |-- Fecha: string (nullable = true)
 |-- Victima: string (nullable = true)
 |-- SubVictima: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Nacionalidad: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Canton: string (nullable = true)
 |-- Distrito: string (nullable = true)
 |-- PoblacionMayor15: integer (nullable = true)
 |-- TasaNetaParticipacion: double (nullable = true)
 |-- TasaOcupacion: double (nullable = true)
 |-- TasaDesempleoAbierto: string (nullable = true)
 |-- PorcentajeEconomicamenteInactivo: double (nullable = true)
 |-- RelacionDependenciaEconomica: double (nullable = true)



In [24]:
datosIntegrados=datosIntegrados.withColumn('TasaDesempleoAbierto',datosIntegrados['TasaDesempleoAbierto'].cast("float").alias('TasaDesempleoAbierto'))
datosIntegrados=datosIntegrados.withColumn('TasaNetaParticipacion',datosIntegrados['TasaNetaParticipacion'].cast("float").alias('TasaNetaParticipacion'))
datosIntegrados=datosIntegrados.withColumn('TasaOcupacion',datosIntegrados['TasaOcupacion'].cast("float").alias('TasaOcupacion'))
datosIntegrados=datosIntegrados.withColumn('PorcentajeEconomicamenteInactivo',datosIntegrados['PorcentajeEconomicamenteInactivo'].cast("float").alias('PorcentajeEconomicamenteInactivo'))
datosIntegrados=datosIntegrados.withColumn('RelacionDependenciaEconomica',datosIntegrados['RelacionDependenciaEconomica'].cast("float").alias('RelacionDependenciaEconomica'))
datosIntegrados=datosIntegrados.withColumn('Fecha',to_date(unix_timestamp(col('Fecha'), 'yyyy-MM-dd').cast("timestamp")))

In [25]:
datosIntegrados.printSchema()

root
 |-- Delito: string (nullable = true)
 |-- SubDelito: string (nullable = true)
 |-- Hora: string (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Victima: string (nullable = true)
 |-- SubVictima: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Nacionalidad: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Canton: string (nullable = true)
 |-- Distrito: string (nullable = true)
 |-- PoblacionMayor15: integer (nullable = true)
 |-- TasaNetaParticipacion: float (nullable = true)
 |-- TasaOcupacion: float (nullable = true)
 |-- TasaDesempleoAbierto: float (nullable = true)
 |-- PorcentajeEconomicamenteInactivo: float (nullable = true)
 |-- RelacionDependenciaEconomica: float (nullable = true)



In [26]:
datosIntegrados.show()

+------+-------------+-------------------+----------+-----------+----------+--------------------+-------------+------------+----------+----------+-------------+----------------+---------------------+-------------+--------------------+--------------------------------+----------------------------+
|Delito|    SubDelito|               Hora|     Fecha|    Victima|SubVictima|                Edad|       Genero|Nacionalidad| Provincia|    Canton|     Distrito|PoblacionMayor15|TasaNetaParticipacion|TasaOcupacion|TasaDesempleoAbierto|PorcentajeEconomicamenteInactivo|RelacionDependenciaEconomica|
+------+-------------+-------------------+----------+-----------+----------+--------------------+-------------+------------+----------+----------+-------------+----------------+---------------------+-------------+--------------------+--------------------------------+----------------------------+
|ASALTO|  ARMA BLANCA|18:00:00 - 20:59:59|2011-09-19|EDIFICACION|      CAFÉ| INTERNET [EDIFIC...|Mayor de eda

In [27]:
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "postgresql-42.1.4.jar") \
    .config("spark.executor.extraClassPath", "postgresql-42.1.4.jar") \
    .getOrCreate()

In [28]:
def mandaPostgres(dataframe, nombre):
    dataframe \
        .write \
        .format("jdbc") \
        .mode('overwrite') \
        .option("url", "jdbc:postgresql://localhost/") \
        .option("user", "postgres") \
        .option("password", "password") \
        .option("dbtable", nombre) \
        .save()

In [29]:
mandaPostgres(datosIntegrados.select("Delito","SubDelito","Hora","Fecha","Victima","SubVictima","Edad","Genero","Nacionalidad","Provincia","Canton","Distrito","PoblacionMayor15","TasaNetaParticipacion","TasaOcupacion","TasaDesempleoAbierto","PorcentajeEconomicamenteInactivo","RelacionDependenciaEconomica"),"prueba")


In [162]:
sqlContext.registerDataFrameAsTable(datosIntegrados,"datosIntegrados")
def top10Distritos():
    return sqlContext.sql("SELECT DISTINCT distrito, COUNT(delito) as Cantidad_Delitos, SUM(tasaocupacion)/COUNT(delito) as Tasa_Ocupacion FROM datosIntegrados group by distrito order by 2 DESC LIMIT 10") 

In [163]:
top10Distritos=top10Distritos()

In [166]:
top10Distritos.show()

+------------+----------------+------------------+
|    distrito|Cantidad_Delitos|    Tasa_Ocupacion|
+------------+----------------+------------------+
|desamparados|               9| 55.36666742960612|
|    san jose|               7| 53.10000010899135|
|   san pablo|               6| 49.29999923706055|
|    alajuela|               6| 52.86666742960612|
|     heredia|               6|54.833333333333336|
|  san carlos|               4| 50.70000076293945|
|  puntarenas|               3| 48.36666742960612|
|    la union|               2|             49.25|
|      quepos|               2|              54.0|
|      grecia|               2|              53.0|
+------------+----------------+------------------+



In [179]:
def delitosDia():
    return sqlContext.sql("SELECT distrito,COUNT(delito) as Cantidad_Delitos,fecha, date_format(fecha, 'EEEE') as Dia_Semana FROM datosIntegrados WHERE distrito='desamparados' group by distrito, fecha ") 

In [180]:
delitosDia=delitosDia()

In [181]:
delitosDia.show()

+------------+----------------+----------+----------+
|    distrito|Cantidad_Delitos|     fecha|Dia_Semana|
+------------+----------------+----------+----------+
|desamparados|               3|2011-06-30|  Thursday|
|desamparados|               3|2011-07-22|    Friday|
|desamparados|               3|2011-03-26|  Saturday|
+------------+----------------+----------+----------+



In [184]:
def delitosTipo():
    return sqlContext.sql("SELECT distrito,COUNT(delito) as Cantidad_Delitos, delito as Tipo_Delito FROM datosIntegrados WHERE distrito='desamparados' group by distrito,delito ") 

In [185]:
delitosTipo=delitosTipo()

In [186]:
delitosTipo.show()

+------------+----------------+-----------+
|    distrito|Cantidad_Delitos|Tipo_Delito|
+------------+----------------+-----------+
|desamparados|               6|     ASALTO|
|desamparados|               3|       ROBO|
+------------+----------------+-----------+



In [220]:
def delitosSexo():
    return sqlContext.sql("SELECT nacionalidad, COUNT(delito) as Cantidad_Delitos FROM datosIntegrados group by nacionalidad") 

In [221]:
delitosSexo=delitosSexo()

In [222]:
delitosSexo.show()

+------------+----------------+
|nacionalidad|Cantidad_Delitos|
+------------+----------------+
|       MUJER|               9|
|      HOMBRE|              54|
+------------+----------------+



In [227]:
def subdelitosCant():
    return sqlContext.sql("SELECT Subdelito, COUNT(delito) as Cantidad_Delitos FROM datosIntegrados group by Subdelito") 

In [228]:
subdelitosCant=subdelitosCant()

In [229]:
subdelitosCant.show()

+--------------------+----------------+
|           Subdelito|Cantidad_Delitos|
+--------------------+----------------+
|         ARMA BLANCA|               4|
|ARDID PREVIO/DIST...|               1|
|       ARMA DE FUEGO|              28|
|        POR DESCUIDO|               6|
|             BOQUETE|               2|
|       POR CONFIANZA|               2|
|     QUITAN CELOSIAS|               7|
|           FORZADURA|              13|
+--------------------+----------------+

